In [62]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import locale
locale.setlocale(locale.LC_ALL, 'ID')
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [54]:
# https://indeks.kompas.com/news/2018-07-19/2

In [55]:
def getIndeksLink(links, page, cat_link, category, date=datetime.strftime(datetime.today(), '%Y-%m-%d')):
    """
    Untuk mengambil seluruh url 
    link pada indeks category tertentu
    date format : YYYY-mm-dd
    """
    print("page ", page)
    if cat_link == 'otomotif':
        url = "https://indeks.kompas.com/"+cat_link+"/"+date+"/"+str(page)
    else :
        url = "https://indeks.kompas.com/"+cat_link+"/"+date+"/"+str(page)
    print(url)
    # Make the request and create the response object: response
    response = requests.get(url)
    # Extract HTML texts contained in Response object: html
    html = response.text
    # Create a BeautifulSoup object from the HTML: soup
    soup = BeautifulSoup(html, "html5lib")
    contentDiv = soup.findAll('div', class_="latest--indeks mt2 clearfix")
    for post in contentDiv:
        link = [post.find('a', href=True)['href'], category]
        links.append(link)
        
    el_page = soup.find('div', class_="paging__wrap clearfix")
    if el_page:
        a_page = soup.find('div', class_="paging__wrap clearfix").findAll('div', class_='paging__item')[-1].find('a')
        if 'data-ci-pagination-page' in a_page:
            max_page = page
        else:
            max_page = int(a_page['data-ci-pagination-page'].replace('\n', '').strip(' '))
            
        if page < max_page:
            links = getIndeksLink(links, page+1, cat_link, category, date)
        
    return links

In [56]:
a_page = soup.find('div', class_="paging__wrap clearfix").findAll('div', class_='paging__item')[-1].find('a')
a_page

<a class="paging__link paging__link--next" data-ci-pagination-page="2" href="http://indeks.kompas.com/otomotif/2018-07-27/2" rel="next">Next</a>

In [227]:
def getDetailBerita(url):
    articles = {}
    for link in links:
        #link
#         articles['url'] = link[0]
        response = requests.get(url)
        html = response.text
        # Create a BeautifulSoup object from the HTML: soup
        soup = BeautifulSoup(html, "html5lib")
        
        #articleid
        articles['id'] = int(soup.find("meta", attrs={'name':'cXenseParse:articleid'})['content'])
        
        #category
#         articles['category'] = link[1]
#         articles['url'] = url
        
        #extract subcategory from breadcrumb
        bc = soup.find('ul', class_="breadcrumb__wrap")
        articles['subcategory'] = bc.findAll('li')[2].text
        
        #article
        article = soup.find("div", class_="read__content")
        
        #extract date
        pubdate = soup.find("meta", attrs={"name":"content_PublishedDate"})['content']
        pubdate = pubdate.strip(' \t\n\r')
        articles['pubdate']=datetime.strftime(datetime.strptime(pubdate, "%Y-%m-%d %H:%M:%S"), "%Y-%m-%d %H:%M:%S")
        
        #extract author
        articles['author'] = soup.find('div', class_="read__author").text
        
        #extract title
        articles['title'] = soup.find('h1', class_="read__title").text      

        #source
        articles['source'] = 'kompas'
        
        #extract comments count
        #articles['comments'] = int(soup.find('a', class_="komentar").find('span').text.replace('Komentar', '').strip(' \t\n\r'))
        
        #extract tags
        tags = soup.find("meta", attrs={'name':'content_tag'})['content']
        article['tags'] = tags
        
        #extract images
        articles['images'] = soup.find('meta', attrs={'property':'og:image'})['content']
        
       
        #hapus link sisip
#         for link in detail.findAll('table', class_="linksisip"):
#             link.decompose()
        
        #hapus video sisip
#         for tag in detail.findAll('div', class_="detail_tag"):
#             tag.decompose()
        
        #extract content
        detail = BeautifulSoup(article.decode_contents().replace('<br/>', ' '), "html5lib")
        content = re.sub(r'\n|\t|\b|\r','',detail.text)
        articles['content'] = content
        
        return articles

In [228]:
links = []

In [229]:
links

[]

In [230]:
# detail = getIndeksLink(links, 1, 'otomotif', 'otomotif')

In [231]:
url = "https://nasional.kompas.com/read/2018/07/28/21012971/menurut-pendiri-pks-2019-tak-akan-ganti-presiden"

In [232]:
articles = {}
response = requests.get(url)
    # Extract HTML texts contained in Response object: html
html = response.text
    # Create a BeautifulSoup object from the HTML: soup
soup = BeautifulSoup(html, "html5lib")
# soup

In [233]:
detail = [getDetailBerita(url)]
detail

[None]

In [222]:
pubdate = soup.find("meta", attrs={"name":"content_PublishedDate"})['content']
pubdate = pubdate.strip(' \t\n\r')
articles['pubdate']=datetime.strftime(datetime.strptime(pubdate, "%Y-%m-%d %H:%M:%S"), "%Y-%m-%d %H:%M:%S")
articles['pubdate']

'2018-07-28 21:06:49'

In [208]:
tags = soup.find("meta", attrs={'name':'content_tag'})['content']
tags
# articles['tags'] = ','.join([x.text for x in tags])

'Densus 88, terduga teroris'

In [209]:
df = pd.DataFrame(detail)

In [210]:
df

,0
0,None
